In [55]:
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
import numpy as np
import time

def show_testimage(test_image):
    plt.imshow(test_image.squeeze(), cmap="gray")
    plt.title(f"Label: {test_label}")
    plt.show()

# Transformation: Bild in Tensor umwandeln
transform = transforms.Compose([transforms.ToTensor()])
# Laden des Test-Datensatzes
test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)
# Zugriff auf ein Testbild (z. B. das erste Bild)
test_image, test_label = test_dataset[123]  # test_image ist ein Tensor, test_label ist die Ziffer

# Testbild anzeigen
#show_testimage(test_image)

# Bild als Vektor p
p_vec = test_image.view(784)
p_vec = p_vec.detach().cpu().numpy() if isinstance(p_vec, torch.Tensor) else np.array(p_vec)

# Implementieren Sie die Vorwärtsrechnung des Neuronalen Netzwerkes 
# o = F(p, v, w, a, b) und messen Sie die Zeit, die für die Berechnung benötigt wird!

# Bias und Weights erzeugen
v_vec = np.random.rand(784,10)
w_vec = np.random.rand(784,784)
a_vec = np.random.rand(784)
b_vec = np.random.rand(10)

start = time.time()

# Layer berechnen
h_vec = np.zeros(784)
h_vec = np.dot(p_vec, w_vec) + a_vec
h_vec = np.maximum(0, h_vec)  # ReLU
o_vec = np.dot(h_vec, v_vec) + b_vec

end = time.time()
print(o_vec)
print(end - start)

[18570.40472847 19359.19702213 19389.0047517  18023.30991097
 19749.80542712 18493.26693958 18558.58218251 18234.23996759
 19272.05500264 19325.69555466]
0.003583192825317383


In [62]:
# Berechnen Sie die Ableitung des Netzwerkes nach einem Parameter vij im
# Forward-Mode mit ihren differenzierbaren Datentypen aus Teil I!

from diffType import diffType

def forward_pass(p_vec, v_vec, w_vec, a_vec, b_vec, i, j):
    
    # Wandlung von p_vec in diffType (Ableitung 0)
    p_vec = [diffType(val, 0.0) for val in p_vec]
    
    # Wandlung von w_vec und a_vec in diffType (Ableitung 0)
    w_vec = [[diffType(val, 0.0) for val in row] for row in w_vec]
    a_vec = [diffType(val, 0.0) for val in a_vec]
    
    # Wandlung von v_vec in diffType, wobei v_ij eine Ableitung von 1 hat
    v_vec = [[diffType(val, 0.0) for val in row] for row in v_vec]
    v_vec[i][j].dvalue = 1.0  # Setzt nur die gewünschte Ableitung auf 1
    
    # Wandlung von b_vec in diffType (Ableitung 0)
    b_vec = [diffType(val, 0.0) for val in b_vec]
    
    # Layer-Berechnung
    h_vec = [sum(p * w for p, w in zip(p_vec, w_row)) + a for w_row, a in zip(w_vec, a_vec)]
    h_vec = [diffType(max(0, h.value), h.dvalue if h.value > 0 else 0.0) for h in h_vec]  # ReLU
    o_vec = [sum(h * v for h, v in zip(h_vec, v_row)) + b for v_row, b in zip(zip(*v_vec), b_vec)]
    
    return o_vec  

i, j = 5, 2  # Beispielhafter Index für v_ij
start = time.time()
result = forward_pass(p_vec, v_vec, w_vec, a_vec, b_vec, i, j)
end = time.time()
# Ausgabe der Ableitung von o nach v_ij
print("Ableitungen:", [o.dvalue for o in result])
print(f"Zeit: {end-start:.3f} Sekunden")


Ableitungen: [np.float64(0.0), np.float64(0.0), np.float64(49.52416218056674), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)]
Zeit: 1.171 Sekunden


In [64]:
# Schätzen Sie, wie lange es dauern würde, den gesamten Gradientenvektor, d.h. alle partiellen Ableitungen von F
# nach allen Parametern vij, wjk, aj, bk im Forward-Mode zu berechnen, und erstellen Sie eine Hochrechnung, 
# wie lange das Training mit dem MNIST-Datensatz über 1000 Epochen dauern würde!

# 784²+7840+2×784 = 624064 Diff. notwendig
# Ausführungszeit oben ca. 1.171 Sekunden
# Gesamt also 730778,944 Sekunden 
# bzw. ca. 203h
# Über 1000 Epochen = 1000 Schritte in Gradientenabstieg
# demnach ca. 2030h +-


In [68]:
import torch
import torch.nn.functional as F

def forward_pass_torch(p_vec, v_vec, w_vec, a_vec, b_vec):
    """
    Berechnet die Vorwärtspropagation mit PyTorch
    """
    p_vec = torch.tensor(p_vec, dtype=torch.float32, requires_grad=False)
    v_vec = torch.tensor(v_vec, dtype=torch.float32, requires_grad=True)  # Ableitung nach v
    w_vec = torch.tensor(w_vec, dtype=torch.float32, requires_grad=True)  # Ableitung nach w
    a_vec = torch.tensor(a_vec, dtype=torch.float32, requires_grad=True)
    b_vec = torch.tensor(b_vec, dtype=torch.float32, requires_grad=True)
    
    h_vec = torch.relu(torch.matmul(p_vec, w_vec) + a_vec)  # Hidden Layer mit ReLU
    o_vec = torch.matmul(h_vec, v_vec) + b_vec  # Output Layer
    return o_vec, v_vec, w_vec, a_vec, b_vec

def compute_loss_and_gradients(p_vec, v_vec, w_vec, a_vec, b_vec, target):
    """
    Berechnet die Kreuzentropie-Verlustfunktion und deren Gradienten
    """
    o_vec, v_vec, w_vec, a_vec, b_vec = forward_pass_torch(p_vec, v_vec, w_vec, a_vec, b_vec)
    
    # Softmax und Kreuzentropieverlust
    loss = F.cross_entropy(o_vec.unsqueeze(0), torch.tensor([target], dtype=torch.long))
    
    # Gradienten berechnen
    loss.backward()
    
    return loss.item(), v_vec.grad, w_vec.grad, a_vec.grad, b_vec.grad

# Testdaten erzeugen
p_vec = test_image.view(784).detach().cpu()
v_vec = torch.rand(784, 10, requires_grad=True)
w_vec = torch.rand(784, 784, requires_grad=True)
a_vec = torch.rand(784, requires_grad=True)
b_vec = torch.rand(10, requires_grad=True)

target_label = test_label  # "Truth" welche Zahl gewählt wurde

for i in range(0,5):
    loss, dv, dw, da, db = compute_loss_and_gradients(p_vec, v_vec, w_vec, a_vec, b_vec, target_label)
    print("Loss:", loss)
    print("Gradienten von v:", dv)

Loss: 1355.33984375
Gradienten von v: tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000, 45.6069,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, 54.0785,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, 43.4496,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, 48.5394,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, 50.4066,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, 45.0292,  0.0000]])
Loss: 1355.33984375
Gradienten von v: tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000, 45.6069,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, 54.0785,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, 43.4496,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, 48.5394,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, 50.4066,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, 45.0292,  0.0000]])
Loss: 1355.33984375
Gradienten von v: tensor([[ 0.0000, 

/tmp/ipykernel_32176/3121635623.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  p_vec = torch.tensor(p_vec, dtype=torch.float32, requires_grad=False)
/tmp/ipykernel_32176/3121635623.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v_vec = torch.tensor(v_vec, dtype=torch.float32, requires_grad=True)  # Ableitung nach v
/tmp/ipykernel_32176/3121635623.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w_vec = torch.tensor(w_vec, dtype=torch.float32, requires_grad=True)  # Ableitung nach w
/tmp/ipykernel_32176/3121635623.py:11: User

In [69]:
import torch
import torch.nn.functional as F

def forward_pass_torch(p_vec, v_vec, w_vec, a_vec, b_vec):
    """
    Berechnet die Vorwärtspropagation mit PyTorch
    """
    p_vec = torch.tensor(p_vec, dtype=torch.float32, requires_grad=False)
    
    h_vec = torch.relu(torch.matmul(p_vec, w_vec) + a_vec)  # Hidden Layer mit ReLU
    o_vec = torch.matmul(h_vec, v_vec) + b_vec  # Output Layer
    return o_vec

def compute_loss(p_vec, v_vec, w_vec, a_vec, b_vec, target):
    """
    Berechnet die Kreuzentropie-Verlustfunktion
    """
    o_vec = forward_pass_torch(p_vec, v_vec, w_vec, a_vec, b_vec)
    loss = F.cross_entropy(o_vec.unsqueeze(0), torch.tensor([target], dtype=torch.long))
    return loss

# Testdaten erzeugen
p_vec = test_image.view(784).detach().cpu()
v_vec = torch.rand(784, 10, requires_grad=True)
w_vec = torch.rand(784, 784, requires_grad=True)
a_vec = torch.rand(784, requires_grad=True)
b_vec = torch.rand(10, requires_grad=True)

target_label = 2  # Beispielhafte Zielklasse
learning_rate = 0.01

# Gradientenabstieg
for epoch in range(5):  # 5 Iterationen
    loss = compute_loss(p_vec, v_vec, w_vec, a_vec, b_vec, target_label)
    
    # Gradienten berechnen
    loss.backward()
    
    # Parameter aktualisieren
    with torch.no_grad():
        v_vec -= learning_rate * v_vec.grad
        w_vec -= learning_rate * w_vec.grad
        a_vec -= learning_rate * a_vec.grad
        b_vec -= learning_rate * b_vec.grad
        
        # Gradienten zurücksetzen
        v_vec.grad.zero_()
        w_vec.grad.zero_()
        a_vec.grad.zero_()
        b_vec.grad.zero_()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.0
Epoch 2, Loss: 0.0
Epoch 3, Loss: 0.0
Epoch 4, Loss: 0.0
Epoch 5, Loss: 0.0


/tmp/ipykernel_32176/2456677009.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  p_vec = torch.tensor(p_vec, dtype=torch.float32, requires_grad=False)
